# How to use `get_rm_from_skycoord`

This example script shows how to get ionospheric rotation measures for a give source,
observer location and array of times. This script uses the default ionospheric settings,
which are good for most purposes.

In [ ]:
from __future__ import annotations

import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.coordinates import EarthLocation, SkyCoord
from astropy.time import Time
from astropy.visualization import quantity_support, time_support
from spinifex import get_rm

In [ ]:
### Required to load local data for example - not needed for normal use
from importlib import resources

with resources.as_file(resources.files("spinifex.data.tests")) as datapath:
    spinifex_data = datapath
###

In [ ]:
times = Time("2020-01-08T01:00:00") + np.arange(10) * 25 * u.min
# create source object
source = SkyCoord(ra=350.85 * u.deg, dec=58.815 * u.deg)
# create Earth Location
lon = 6.367 * u.deg
lat = 52.833 * u.deg
dwingeloo = EarthLocation(lon=lon, lat=lat, height=0 * u.km)
# get rotation measures
rm = get_rm.get_rm_from_skycoord(
    loc=dwingeloo,
    times=times,
    source=source,
    # We set these options to use the data packaged with spinifex
    # Unsetting them will cause the function to download the data from the internet
    prefix="cod",
    output_directory=spinifex_data,
)

In [ ]:
# print to screen
rotation_measures = rm.rm
rm_times = rm.times
elevations = rm.elevation
azimuths = rm.azimuth
print("time      RM (rad/lambda^2)      azimuth (deg)      elevation (deg)")
for myrm, tm, az, el in zip(rotation_measures, rm_times, azimuths, elevations):
    print(f"{tm.isot} {myrm} {az:3.2f} {el:2.2f}")

In [ ]:
with time_support(), quantity_support():
    fig, ax = plt.subplots()
    im = ax.errorbar(rm.times.datetime, rm.rm, rm.rm_error, fmt=".")
    ax.set_ylabel("RM / rad/m$^2$")
    plt.xticks(rotation=20)